In [5]:
# https://www.quantamagazine.org/magnetism-may-have-given-life-its-molecular-asymmetry-20230906/

In [6]:
#Magnetism May Have Given Life Its Molecular Asymmetry
#The preferred “handedness” of biomolecules could have emerged from biased interactions between electrons and 
#magnetic surfaces, new research suggests.

In [2]:
import yfinance as yf
import pandas as pd

def calculate_ciss_effect(stock_data):
    # Initialize parameters
    spin_up = spin_down = 0
    trend = ["Bearish"]  # Assume starting with a Bearish trend

    # Compute the spin for each day
    for i in range(1, len(stock_data)):
        if stock_data[i] > stock_data[i - 1]:
            spin_up += 1
        else:
            spin_down += 1

        if spin_up > spin_down:
            trend.append("Bearish")
        else:
            trend.append("Bullish")

    return trend

def simulate_trades(stock_data):
    trends = calculate_ciss_effect(stock_data['Close'])
    positions = []
    cash = 10000  # Starting cash
    shares = 0
    buy_price = 0
    portfolio_value = cash
    peak_portfolio = cash
    max_drawdown = 0
    cumulative_change = 0

    for i in range(1, len(stock_data)):
        current_price = stock_data['Close'][i]
        current_trend = trends[i]
        last_trend = trends[i - 1]

        # Buy condition: Trend changes from Bearish to Bullish
        if last_trend == "Bearish" and current_trend == "Bullish":
            if shares == 0:  # Only buy if not currently holding shares
                shares = cash // current_price
                buy_price = current_price
                cash -= shares * current_price
                buy_date = stock_data.index[i]

        # Sell condition: Trend changes from Bullish to Bearish
        elif last_trend == "Bullish" and current_trend == "Bearish":
            if shares > 0:  # Only sell if currently holding shares
                cash += shares * current_price
                sell_date = stock_data.index[i]
                percent_change = ((current_price - buy_price) / buy_price) * 100
                cumulative_change += percent_change
                portfolio_value = cash
                max_drawdown = min(max_drawdown, (portfolio_value - peak_portfolio) / peak_portfolio * 100)
                peak_portfolio = max(peak_portfolio, portfolio_value)

                positions.append({
                    'Buy Date': buy_date,
                    'Buy Price': buy_price,
                    'Sell Date': sell_date,
                    'Sell Price': current_price,
                    'Qty': shares,
                    'Change (%)': percent_change,
                    'Cumulative Change (%)': cumulative_change,
                    'Portfolio After Trade': portfolio_value,
                    'Max Drawdown (%)': max_drawdown
                })
                shares = 0  # Reset shares after selling

    # Check if a position is still open at the end of the period
    if shares > 0:
        positions.append({
            'Buy Date': buy_date,
            'Buy Price': buy_price,
            'Sell Date': 'Position still open',
            'Sell Price': current_price,
            'Qty': shares,
            'Change (%)': ((current_price - buy_price) / buy_price) * 100,
            'Cumulative Change (%)': cumulative_change,
            'Portfolio After Trade': cash + shares * current_price,
            'Max Drawdown (%)': max_drawdown
        })

    return pd.DataFrame(positions)

# Define the ticker symbol and date range
ticker = 'RELIANCE.NS'
start = '2024-01-01'
end = '2024-05-01'

# Download the stock data
try:
    stock_data = yf.download(ticker, start=start, end=end)
    # Simulate trades based on CISS effect
    trade_summary = simulate_trades(stock_data)
    #print(trade_summary)
except Exception as e:
    print(f"Failed to download or process the data: {str(e)}")
round(trade_summary,2)

[*********************100%%**********************]  1 of 1 completed


C:\Users\DELL\AppData\Local\Temp\ipykernel_5628\2912409517.py:11: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if stock_data[i] > stock_data[i - 1]:
C:\Users\DELL\AppData\Local\Temp\ipykernel_5628\2912409517.py:35: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  current_price = stock_data['Close'][i]


,Buy Date,Buy Price,Sell Date,Sell Price,Qty,Change (%),Cumulative Change (%),Portfolio After Trade,Max Drawdown (%)
0,2024-01-03,2583.3,2024-01-04,2596.65,3.0,0.52,0.52,10040.05,0
1,2024-01-09,2580.5,2024-01-10,2650.10,3.0,2.70,3.21,10248.85,0
